In [28]:
# Config
import copy
import os
import numpy as np
from numpy.linalg import norm
import matplotlib
import matplotlib.pyplot as plt

# Path for ffmpeg (if animations are needed)
plt.rcParams['animation.ffmpeg_path'] = "C:\\Users\\Fredric\\Documents\\ffmpeg\\ffmpeg-n4.4-latest-win64-gpl-4.4\\bin\\ffmpeg.exe"

# Path for .pkl files output
target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\tungurahua\\"
target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\conduit1D\\"


# Path for Quail source code
source_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\src\\"

# Path for Quail entry point
quail_path = os.path.join(source_dir, "quail")

# Name of file to run (must be located in target_dir)
target_file = "conduit_p.py"

In [31]:
%load_ext autoreload
%autoreload 2

# Import quail modules
os.chdir(source_dir)
import argparse
import importlib
import sys
import scipy

import defaultparams as default_deck
import errors
from general import ShapeType, SolverType, PhysicsType

import meshing.common as mesh_common
import meshing.gmsh as mesh_gmsh
import meshing.tools as mesh_tools

import numerics.helpers.helpers as helpers
import numerics.timestepping.tools as stepper_tools

import physics.zerodimensional.zerodimensional as zerod
import physics.euler.euler as euler
import physics.navierstokes.navierstokes as navierstokes
import physics.scalar.scalar as scalar
import physics.chemistry.chemistry as chemistry
import physics.multiphasevpT.multiphasevpT as multiphasevpT

import processing.readwritedatafiles as readwritedatafiles
import processing.post as post
import processing.plot as plot
import processing
import processing.mdtools as mdtools

import solver.DG as DG
import solver.ADERDG as ADERDG
import solver.tools as solver_tools

import time
import multiprocessing as mp
from multidomain import Domain, Observer

os.chdir(target_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
solver1D_from = lambda i: readwritedatafiles.read_data_file(
  #f"featuretest_conduit1_{i}.pkl")
  f"conduitSteadyState_inlet_cont_{i}.pkl")
solver = solver1D_from(80)


In [33]:
solver.state_coeffs.shape

(200, 2, 7)

In [41]:
(A-B)[0:6,0:6].todense().astype(np.float16)

array([[-0.5,  0.5,  0. ,  0. ,  0. ,  0. ],
       [-0.5, -0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. , -0.5,  0. ,  0.5,  0. ,  0. ],
       [ 0. ,  0. , -0.5, -0. ,  0.5,  0. ],
       [ 0. ,  0. ,  0. , -0.5,  0. ,  0.5],
       [ 0. ,  0. ,  0. ,  0. , -0.5, -0. ]], dtype=float16)

In [34]:
from scipy.sparse import dok_array

ne = 120
nb = 2
N = ne*nb
BC_index = N-1

A = dok_array((N, N,))

for i in range(ne):
    A[nb*i, nb*i] += -0.5
    A[nb*(i+1)-1, nb*(i+1)-1] += 0.5
    if i > 0:
        A[nb*i, nb*i-1] += -0.5
    if i < ne-1:
        A[nb*(i+1)-1, nb*(i+1)] += 0.5
# Check for one-sided numerical fluxes where BC is absent
if BC_index == 0:
  A[-1,-1] += 0.5
elif BC_index == N-1:
  A[0,0] += -0.5
A = A.tocsr()

''' Assemble interior flux matrix B '''
u = np.einsum('jn, jm, ijm -> ijn', 
  solver.basis.get_values(solver.elem_helpers.quad_pts),
  solver.elem_helpers.quad_wts,
  solver.elem_helpers.djac_elems)
# [ne, nq, nb1, ndims] x [ne, nq, nb2] -> [ne, nb1, nb2]
B_vec = np.einsum('ijml, ijn -> imn',
  solver.elem_helpers.basis_phys_grad_elems,
  u)
B = dok_array((N, N,))
for i in range(ne):
  B[nb*i:nb*(i+1), nb*i:nb*(i+1)] = B_vec[i,:,:]
B = B.tocsr()

''' Assemble interior mass matrix M '''
# [nq, nb] x [nq, 1] x [ne, nq, 1] -> [ne, nq, nb]
u = np.einsum('jn, jm, ijm -> ijn', 
  solver.basis.get_values(solver.elem_helpers.quad_pts),
  solver.elem_helpers.quad_wts,
  solver.elem_helpers.djac_elems)
# [nq, nb1] x [ne, nq, nb2] -> [ne, nb1, nb2]
M_vec = np.einsum('jm, ijn -> imn',
  solver.basis.get_values(solver.elem_helpers.quad_pts),
  u)
M = dok_array((N, N,))
nb = nb
for i in range(ne):
  M[nb*i:nb*(i+1), nb*i:nb*(i+1)] = M_vec[i,:,:]
M = M.tocsr()

''' Set barotropic state, state gradient '''
# Function that maps p to the full state
path_U = lambda p: p
# Function that maps p to drho/dp
path_drhodp = lambda p: np.ones_like(p)


In [12]:
p = np.ones((ne*nb))
p.shape, A.shape

((360,), (360, 360))

In [16]:
p - scipy.sparse.linalg.spsolve(A-B, (A-B)@p)

array([-1.110223024625157e-15, -8.881784197001252e-16,
       -1.110223024625157e-15, -8.881784197001252e-16,
       -8.881784197001252e-16, -8.881784197001252e-16,
       -8.881784197001252e-16, -8.881784197001252e-16,
       -8.881784197001252e-16, -6.661338147750939e-16,
       -6.661338147750939e-16, -6.661338147750939e-16,
       -8.881784197001252e-16, -6.661338147750939e-16,
       -8.881784197001252e-16, -6.661338147750939e-16,
       -6.661338147750939e-16, -6.661338147750939e-16,
       -8.881784197001252e-16, -6.661338147750939e-16,
       -8.881784197001252e-16, -6.661338147750939e-16,
       -6.661338147750939e-16, -6.661338147750939e-16,
       -8.881784197001252e-16, -6.661338147750939e-16,
       -8.881784197001252e-16, -6.661338147750939e-16,
       -6.661338147750939e-16, -6.661338147750939e-16,
       -8.881784197001252e-16, -6.661338147750939e-16,
       -8.881784197001252e-16, -6.661338147750939e-16,
       -6.661338147750939e-16, -6.661338147750939e-16,
       -8.

In [ ]:

''' Compute initial guess from average weight '''
gsource = [s for s in solver.physics.source_terms if
  type(s) is GravitySource][0]
# Barotropic gravity source function [ne, nq]
g_fn = lambda p: gsource.get_source(
    self.solver.physics,
    path_U(p),
    self.solver.elem_helpers.x_elems,
    self.solver.time)[
      :,:,self.solver.physics.get_state_index("XMomentum")]
vec_cast = lambda S: np.expand_dims(S.ravel(),axis=1)

# Build source function
S_fn = lambda p: g_fn(p)
# Add traction
if traction_fn is not None:
  S_fn = lambda p: g_fn(p) \
    + traction_fn(self.solver.elem_helpers.x_elems.squeeze(axis=2))

# Define load vector function
f_fn = lambda p: -b + M @ vec_cast(S_fn(p)) + delta_source

''' Compute initial guess '''
rho0 = np.sum(
  solver.state_coeffs[:,:,solver.physics.get_mass_slice()],axis=2) \
  .ravel()[BC_index]
p_like = 0.0*b.todense() + 1.0
f = -b + (-rho0*gsource.gravity) * M @ p_like + delta_source
p = scipy.sparse.linalg.spsolve(A-B, f)
p = np.expand_dims(p, axis=1)
# Copy for internal debugging
p_guess = p

''' Fixed point iteration '''
# Set relaxation parameter for P0
mu = 0.0
# if solver.order == 0:
  # mu = 0
# Regularization matrix
# R = 0*scipy.sparse.identity(self.N).todense()
# R[29,29] = -1
# R[29,30] = 1
# R[30,29] = -1
# R[30,30] = 1
# (0.5*R@p_guess).T - delta_source

# Simple fixed point iteration obtained from inverting the linear part of
# the equation (A-B)*p = f(p) -> p_{k+1} = (A-B) \ f(p_{k})
fixedpointiter = lambda p: scipy.linalg.solve(A-B+0*scipy.sparse.identity(self.N)+mu*M,
  f_fn(p) + mu*M@p + 0*p)
evalresidual = lambda p : np.linalg.norm(
  (A-B)@p - f_fn(p), 'fro')
# Newton iteration sending residual of equation to zero
newtoniter = lambda p: p - scipy.linalg.solve(
  A-B+gsource.gravity*M@np.diag(path_drhodp(p).ravel()), # (A-B) - M*diag(d(-rho*g)/dp)
  (A-B)@p - f_fn(p))

residuals = np.array([evalresidual(p)])    
N_iter = self.N_iter
# # Increase allowable for p0
# if solver.order == 0:
#   N_iter = 1

# Start fixed point iteration
# for i in range(2):
for i in range(N_iter):
  p = newtoniter(p)
  fpi_res = evalresidual(p)
  # print(fpi_res)
  residuals = np.append(residuals, fpi_res)
  if fpi_res < self.FPI_TOL:
    break

if fpi_res > self.FPI_TOL:
  warnings.warn(f"Fixed point iteration in hydrostatic1D has large residual: {fpi_res}")

# Save residual history
self.residuals = residuals
# Compute difference in pressure between pressure vector and eval'd state
self.pError = np.linalg.norm(p - 
  self.solver.physics.compute_additional_variable("Pressure",
    path_U(p) ,True).ravel()
)
# Evaluate Quail-ready state
U = path_U(p)
# Check unsteady problem residual
try:
  self.unsteady_residuals_face = np.zeros_like(U)
  self.solver.get_boundary_face_residuals(U, self.unsteady_residuals_face)
  self.solver.get_interior_face_residuals(U, self.unsteady_residuals_face)
  self.unsteady_residuals_elements = np.zeros_like(U)
  self.solver.get_element_residuals(U, self.unsteady_residuals_elements)
  self.unsteady_residuals = self.unsteady_residuals_face\
    + self.unsteady_residuals_elements
except KeyError as err:
  # Data not found in boundary data net
  self.logger.info(f'''The following exception occurred due to evaluating
    residuals before boundary data was posted. If the residual is not needed
    for computation and is only used for verification, then this error can
    be safely ignored.''')
  self.logger.error(f"KeyError: {err}",stack_info=True)

# plt.plot(self.x, p)

# Replace solver state coefficients with hydrostatic state
self.solver.state_coeffs = U

# Replace output initial condition with equilibrated condition
if self.solver.params["WriteInitialSolution"]:
  readwritedatafiles.write_data_file(self.solver, 0)